In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(1337)

In [3]:
import v5_practice1 as vp

In [6]:
with Path("../input.txt").open("r", encoding="utf-8") as f:
    text = f.read()

block_size = 8
batch_size = 32
n_heads = 4
n_embd = 32
max_iters = 4000
eval_iters = 200

data = vp.Data(text)
data.block_size = block_size
data.batch_size = batch_size
data.n_embd = n_embd

In [7]:
# print("".join(data.decode(data.encode("hi there"))))
# xb, yb = data.get_batch('train')
# print(xb, yb)

In [8]:
# m = vp.LanguageModel(n_embd, block_size, data.vocab_size, n_heads)
# out = m.generate(torch.zeros((1, 1), dtype=torch.long))
# # out
# print("".join(data.decode(out[0].tolist())))

In [ ]:
m = vp.LanguageModel(n_embd, block_size, data.vocab_size, n_heads)
m = m.to(DEVICE)
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
for iter_ in range(max_iters):
    if iter_ % eval_iters == 0:
        out = data.estimate_loss(m, eval_iters)
        print(f"train loss {out['train']} val loss {out['val']}")
    xb, yb = data.get_batch('train')
    logits, losses = m(xb, yb)
    optimizer.zero_grad()
    losses.backward()
    optimizer.step()

out = m.generate(torch.zeros((1, 1), dtype=torch.long))
print("".join(data.decode(out[0].tolist())))

train loss 4.314807891845703 val loss 4.313204288482666
train loss 2.6730310916900635 val loss 2.674984931945801
train loss 2.4953665733337402 val loss 2.50203537940979
train loss 2.427274465560913 val loss 2.4537932872772217
train loss 2.3918449878692627 val loss 2.3824310302734375
train loss 2.3352694511413574 val loss 2.347018241882324
train loss 2.3211679458618164 val loss 2.321964979171753
train loss 2.277582883834839 val loss 2.30521297454834
